In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re

In [5]:
articles = pd.read_csv('/kaggle/input/corpus/data.csv')
print(articles.head())

                                                data
0  named entity real world object assigned name e...
1  sql stand structured query language programmin...
2  although blockchain still infancy opportunity ...
3  job interview scary fresher especially attendi...
4  memcached highly performant distributed cachin...


In [6]:
data = list(articles['data'])
tagged_Data = [TaggedDocument(words=word_tokenize(_d.lower()), tags = [str(i)]) for i, _d in enumerate(data)]

In [7]:
model = Doc2Vec(vector_size=50,min_count=10,epochs=50)

In [9]:
model.build_vocab(tagged_Data)
k = model.wv.key_to_index.keys()
print(len(k))

205


In [10]:
model.train(tagged_Data,total_examples=model.corpus_count,epochs=model.epochs)
model.save('doc2vec.model')
print("Model Saved")

Model Saved


In [11]:
#pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
resume_path = '/kaggle/input/resume3/resume3.pdf'
resume = ''
pdfReader = PyPDF2.PdfReader(resume_path)
for i in range(len(pdfReader.pages)):
    pageObj = pdfReader.pages[i]
    resume += pageObj.extract_text()

In [17]:
resume = resume.lower()
resume = re.sub('[^a-z]',' ',resume)

In [18]:
def extract_data(url):
    list1 = []
    count = 0
    rsp = requests.get(url)
    if rsp.status_code == 200:
        soup = BeautifulSoup(rsp.text,'html.parser')
        l = soup.find(class_='av-company-description-page mb-2')
        web = ''.join([i.text for i in l.find_all(['p','li'])])
        list1.append(web)
        return web
    else:
        print("Error")

In [25]:
# Sample list of links
jd_links = [
    'https://datahack.analyticsvidhya.com/jobathon/clix-capital/senior-manager-growthrisk-analytics-2',
    'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-growth-analytics-2',
    'https://datahack.analyticsvidhya.com/jobathon/clix-capital/manager-risk-analytics-2',
    'https://datahack.analyticsvidhya.com/jobathon/cropin/data-scientist-85'
]

jd_df = pd.DataFrame(columns=['links', 'data'])
jd_df['links'] = jd_links

# Extracting the JD data from the links:
for i in range(len(jddf)):
    jd_df['data'][i] = extract_data(jd_df['links'][i])


In [36]:

jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : str.lower(x))


jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : " ".join(re.findall('[\w]+',x))
)

jd_df.loc[:,"data"] = jd_df.data.apply(lambda x : re.sub(r'\d+','',x))

In [37]:
from numpy.linalg import norm
model = Doc2Vec.load('doc2vec.model')
v1 = model.infer_vector(resume.split())
v2 = model.infer_vector(jd_df['data'][3].split())
cosine_similarity = (np.dot(np.array(v1),np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(cosine_similarity,3))

0.695
